# 4个主要脚本

- run_gym.py: 主函数
- pposgd_simple.py: PPO算法的SGD实现
- mlp_policy.py: 多层感知器策略，主要用于非视觉传感输入
- cnn_policy.py: CNN策略，主要用于视觉传感输入

# 可调参数

## run_gym.py

### argparse:

In [ ]:
def main():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--env', help='environment ID', type=str, default='Pendulum-v0')
    parser.add_argument('--seed', help='RNG seed', type=int, default=None)
    parser.add_argument('--num-timesteps', type=int, default=int(1e6))
    args = parser.parse_args()
    logger.configure()
    train(args.env, num_timesteps=args.num_timesteps, seed=args.seed)

 - --env: 环境标识
 - --seed: 伪随机数生成器初始值，用相同的seed，每次的随机数都会一样
 - --num-timesteps: 每个episode中的一个step算一个timestep，num-timesteps表示所有episode的step之和

### train()函数

In [ ]:
def train(env_id, num_timesteps, seed):
    from baselines.ppo1 import mlp_policy, pposgd_simple
    U.make_session(num_cpu=1).__enter__()
    def policy_fn(name, ob_space, ac_space):
        return mlp_policy.MlpPolicy(name=name, ob_space=ob_space, ac_space=ac_space,
            hid_size=64, num_hid_layers=2)
    def make_gym_env(env_id, seed):
        """
        Create a wrapped, monitored gym.Env for MuJoCo.
        """
        env = gym.make(env_id)
        env = Monitor(env, logger.get_dir())
        if seed is not None:
            set_global_seeds(seed)
            env.seed(seed)
        return env
    env = make_gym_env(env_id, seed)
    pposgd_simple.learn(env, policy_fn,
            max_timesteps=num_timesteps,
            timesteps_per_actorbatch=2048,
            clip_param=0.2, entcoeff=0.00,
            optim_epochs=10, optim_stepsize=3e-4, optim_batchsize=64,
            gamma=0.995, lam=0.97, schedule='constant',
        )
    env.close()

- from baselines.ppo1 import mlp_policy, pposgd_simple
    - pposgd_simple是PPO的算法实现，是必须导入的。
    - 如果是基于视觉的任务，一般会用CNN（卷积神经网络），那么就要导入cnn_policy。
    - 如果是非视觉的、基于传感器的任务，一般会用FFNN（前馈神经网络）,那么就要导入mlp_policy。

- U.make_session(num_cpu=1).__enter__()
    - 给定CPU的数量，这里是1

# 初次编辑日期

2018年5月2日

# 参考文献

[1] https://github.com/openai/baselines